### Imports

In [1]:
from __future__ import division, print_function
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt
import time
import os
import cv2
from PIL import Image, ImageChops

plt.ion()   # interactive mode
%matplotlib inline
%env JOBLIB_TEMP_FOLDER="/tmp"

env: JOBLIB_TEMP_FOLDER="/tmp"


In [2]:
def summary(input_size, model):
    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split('.')[-1].split("'")[0]
            module_idx = len(summary)

            m_key = '%s-%i' % (class_name, module_idx+1)
            summary[m_key] = OrderedDict()
            summary[m_key]['input_shape'] = list(input[0].size())
            summary[m_key]['input_shape'][0] = -1
            summary[m_key]['output_shape'] = list(output.size())
            summary[m_key]['output_shape'][0] = -1

            params = 0
            if hasattr(module, 'weight'):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                if module.weight.requires_grad:
                    summary[m_key]['trainable'] = True
                else:
                    summary[m_key]['trainable'] = False
            if hasattr(module, 'bias'):
                params +=  torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]['nb_params'] = params

        if not isinstance(module, nn.Sequential) and \
           not isinstance(module, nn.ModuleList) and \
           not (module == model):
            hooks.append(module.register_forward_hook(hook))

    # check if there are multiple inputs to the network
    if isinstance(input_size[0], (list, tuple)):
        x = [Variable(torch.rand(1,*in_size)) for in_size in input_size]
    else:
        x = Variable(torch.rand(1,*input_size))

    # create properties
    summary = OrderedDict()
    hooks = []
    # register hook
    model.apply(register_hook)
    # make a forward pass
    model(x)
    # remove these hooks
    for h in hooks:
        h.remove()

    return summary

### Data

In [3]:
TRAIN_DIR = '../data/boneage-training-dataset/'
TRAIN_LABEL = '../data/train.csv'
FILETYPE = '.png'
use_gpu = torch.cuda.is_available()

### Dataset class

In [4]:
class BoneAgeDataset(Dataset):
    """Bone Age dataset."""

    def __init__(self, root_dir, labels, filetype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.filetype = filetype

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.labels.iloc[idx]['id']) + self.filetype)
        img = Image.open(img_name).convert('RGB')
#         image = cv2.imread(img_name)[:,:,::-1]
        label = self.labels.iloc[idx]['boneage'].astype(np.float32)

        if self.transform:
            img = self.transform(img)
            
        sample = {'image': img, 'label': label}

        return sample

### Transforms

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(1024),
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Scale(1024),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ]),
}

### Data loaders

In [6]:
labels = pd.read_csv(TRAIN_LABEL)
bins = xrange(20)
labels['bins'] = np.digitize(labels['boneage'], bins=bins)

In [7]:
skf = StratifiedKFold(n_splits=4)
splits = []
for train, test in skf.split(np.zeros(len(labels)), labels['bins']):
    fold_label = {
        'train': labels.loc[train, ('id', 'boneage')],
        'val': labels.loc[test, ('id', 'boneage')]
    }
    splits.append(fold_label)

/home/user/software/miniconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


In [8]:
fold_label = splits[0]

In [9]:
image_datasets = {x: BoneAgeDataset(TRAIN_DIR, fold_label[x], FILETYPE, data_transforms[x])
                  for x in ['train', 'val']}
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=16)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [10]:
for i_batch, sample in enumerate(dataloders['val']):
    print(i_batch, sample['image'].size())
    print(i_batch, sample['image'].mean())
    if i_batch == 4:
        break

RuntimeError: Traceback (most recent call last):
  File "/home/user/software/miniconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/user/software/miniconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 106, in default_collate
    return {key: default_collate([d[key] for d in batch]) for key in batch[0]}
  File "/home/user/software/miniconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 106, in <dictcomp>
    return {key: default_collate([d[key] for d in batch]) for key in batch[0]}
  File "/home/user/software/miniconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 89, in default_collate
    storage = batch[0].storage()._new_shared(numel)
  File "/home/user/software/miniconda2/lib/python2.7/site-packages/torch/storage.py", line 113, in _new_shared
    return cls._new_using_fd(size)
RuntimeError: unable to wri

In [11]:
class Net(nn.Module):
    def __init__(self, input_shape=(3, 224, 224)):
        super(Net, self).__init__()
        
        resnet = models.resnet18(pretrained=True)
        num_ftrs = resnet.fc.in_features
        resnet.fc = nn.Linear(num_ftrs, 1)

        self.resnet = resnet
#         self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
#         n_size = self._get_conv_output(input_shape)
#         self.fc1 = nn.Linear(n_size, 75)
#         self.fc2 = nn.Linear(75, 1)

    def forward(self, x):
        x = self.resnet(x)
        
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
        return x
    
model = Net([3, 224, 224])
    
# print(summary([3, 224, 224], model))

if use_gpu:
    model.cuda()

In [12]:
lr = 0.01
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [13]:
def ccc(x, y):
#     print('X:', x)
    mean_x = torch.mean(x)
#     print('Mean x:', mean_x)
    mean_y = torch.mean(y)
#     print('Mean y:', mean_y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    xn = torch.norm(xm, 2)
#     print('Xn:', xn)
    yn = torch.norm(ym, 2)
#     print('Yn:', yn)
    xydot = ym.dot(xm.view(-1))
#     print('XYdot:', xydot)

    ccc_num = 2 * xydot / xm.size()[0]
#     print('CCC num:', ccc_num)
    ccc_den = xn + yn + torch.pow(mean_x.sub(mean_y), 2)
#     print('CCC den:', ccc_den)

    ccc = ccc_num / ccc_den
    return ccc
    

def train(epoch, train_loader):
    model.train()
    for batch_idx, sample in enumerate(train_loader):
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.l1_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            c = ccc(output, target)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tCCC: {:.4f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0], c.data.cpu().numpy()[0]))

def test(test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for sample in test_loader:
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        c = ccc(output, target)

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, CCC: {}\n'.format(test_loss, c.data.cpu().numpy()[0]))

In [14]:
for epoch in range(1, 10 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])
    break

Train Epoch: 1 [0/9453 (0%)]	Loss: 131.920242	CCC: 0.0002


error: [Errno 111] Connection refused

In [ ]:
test(dataloders[phase])